## Машинное обучение для текстов

**Термины и определения:**

*Корпус* - набор текстов, в котором эмоции и ключевые слова уже размечены.

*Регулярные выражения* - инструмент для поиска текстов и чисел по шаблону.

*Сентимент-анализ* - анализ тональности текста (позитивный или негативный), выявляет эмоционально окрашенные слова.

*Токенизация* - разбиение текста на токены: отдельные фразы, слова, символы.

*Лемматизация* - приведение слова к начальной форме (лемме).

*Векторизация* - перевод текстов в векторный формат. Близкие или похожие векторы могут отображать похожие слова. Сходство векторов рассчитывается евклидовым расстоянием: чем меньше расстояние, тем сильнее похожи векторы.

**Этапы обработки текстов:**

- Предобработка текста:
  - токенизация;
  - лемматизация;
  - очистка от стоп-слов и ненужных символов (в том числе с помощью регулярных выражений);
  - добавление маркеров начала и конца предложения.
- Токенизация (с помощью моделей, обращающихся к словарям токенов)
- Прогнозирование (с помощью моделей, например для сентимент-анализа)

**Методы векторизации:**

- *"Мешок слов"* - (англ. bag of words) модель, которая преобразует текст в вектор, не учитывая порядок слов.
- *TF-IDF* - оценка важности слов путем подсчета количества их упоминаний в текстах и во всем корпусе.
- *N-граммы* - последовательность из нескольких слов (N - количество слов в составе N-граммы: униграммы, биграммы, триграммы).
- Embeddings - Смысл слов определяется их контекстом. Векторы-эмбеддинги содержат данные о соотношении разных слов и их свойствах.
  - *Word2vec* - предсказывает соседи заданные слова или нет. Слова считаются соседями, если находятся в одном «окне» (максимальном расстоянии между словами). Пара слов — это признаки, а являются ли они соседями — это целевой признак. Задача word2vec — научить модель отличать истинные пары соседей от случайных - решить задачу бинарной классификации, где признаки — это слова, а целевой признак — ответ на вопрос: перед нами истинные слова-соседи или нет.
  - BERT (англ. Bidirectional Encoder Representations from Transformers) — нейронная сеть от Google. При анализе учитывает контекст не только соседних, но и более дальних слов. В анализе текстов применяют предобученную на большом корпусе модель.
  - FastText, GloVe (англ. Global Vectors for Word Representation), ELMO (англ. Embeddings from Language Models), GPT (англ. Generative Pre-Training Transformer) и другие.

In [110]:
# загрузка недостающих библиотек
%pip install pymystem3
%pip install nltk
%pip install torch torchvision torchaudio
%pip install transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
     - -------------------------------------- 0.2/7.2 MB 7.4 MB/s eta 0:00:01
     -- ------------------------------------- 0.5/7.2 MB 6.7 MB/s eta 0:00:01
     ---- ----------------------------------- 0.9/7.2 MB 7.9 MB/s eta 0:00:01
     ------- -------------------------------- 1.3/7.2 MB 8.0 MB/s eta 0:00:01
     --------- ------------------------------ 1.7/7.2 MB 9.0 MB/s eta 0:00:01
     ----------- ---------------------------- 2.1/7.2 MB 9.7 MB/s eta 0:00:01
     -------------- ------------------------- 2.6/7.2 MB 9.8 MB/s eta 0:00:01
     ----------------- ---------------------- 3.1/7.2 MB 10.6 MB/s eta 0:00:01
     -------------------- ------------------- 3.7/7.2 MB 10.9 MB/s eta 0:00:01
     ------------------------ --------------- 4.4/7.2 MB 11.6 MB/s eta 0:00:01
     --------

In [1]:
# подключение библиотек

import pandas as pd

# для работы с путями к файлам
import os
import pathlib
# для поиска регулярных выражений
import re
# для работы с файлами формата .json
import json

# подключение библиотеки PyTorch для работы с BERT
import torch
import transformers

# библиотека для обработки текста (в данном случае для исключения стоп-слов)
import nltk
from nltk.corpus import stopwords

# библиотека для лемматизации - python-оболочка для морфологического анализатора русского языка MyStem 3.1 от Яндекс
from pymystem3 import Mystem

# для векторизации текста (создания мешка слов и N-грамм)
from sklearn.feature_extraction.text import CountVectorizer
# для определения TF-I DF - частоты упоминания слов в тексте
from sklearn.feature_extraction.text import TfidfVectorizer
# логистическая регрессия
from sklearn.linear_model import LogisticRegression

### Выделение лемм в тексте

In [32]:
text = '@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'
model = Mystem()
lemmas = model.lemmatize(text)
print(lemmas)
# вывод обнаруженных в тексте лемм через разделитель |
print('\nСписок лемм через разделитель:', '|'.join(lemmas))
# распечатка полной информации
print ('Полная информация:', json.dumps(model.analyze(text), ensure_ascii=False))

['@', 'first', '_', 'timee', ' ', 'хоть', ' ', 'я', ' ', 'и', ' ', 'школоть', ', ', 'но', ' ', 'поверять', ', ', 'у', ' ', 'мы', ' ', 'то', ' ', 'же', ' ', 'самый', ' :', 'D', ' ', 'общество', ' ', 'профилирующий', ' ', 'предмет', ' ', 'тип', ')\n']

Список лемм через разделитель: @|first|_|timee| |хоть| |я| |и| |школоть|, |но| |поверять|, |у| |мы| |то| |же| |самый| :|D| |общество| |профилирующий| |предмет| |тип|)

Полная информация: [{"text": "@"}, {"analysis": [], "text": "first"}, {"text": "_"}, {"analysis": [], "text": "timee"}, {"text": " "}, {"analysis": [{"lex": "хоть", "wt": 0.6706906113, "gr": "PART="}], "text": "хоть"}, {"text": " "}, {"analysis": [{"lex": "я", "wt": 0.9999716281, "gr": "SPRO,ед,1-л=им"}], "text": "я"}, {"text": " "}, {"analysis": [{"lex": "и", "wt": 0.9999770357, "gr": "CONJ="}], "text": "и"}, {"text": " "}, {"analysis": [{"lex": "школоть", "wt": 0.3767377563, "qual": "bastard", "gr": "V,сов,пе=прош,ед,прич,кр,жен,страд"}], "text": "школота"}, {"text": ", "}

### Очистка текста (удаление ненужных символов)

In [38]:
text = "@first_timee хоть я и школота, но поверь, у мы то же самый :d общество профилировать предмет типа)"
# замена в тексте всех символов, кроме русских букв и пробелов на пробелы
print(re.sub(r'[^а-яА-ЯёЁ ]', ' ', text))
# чтобы убрать лишние пробелы добавляем разделение по пробелам (.split) и обратное соединение через один пробел(.join)
print(' '.join(re.sub('[^а-яА-ЯёЁ ]', ' ', text).split()))

             хоть я и школота  но поверь  у мы то же самый    общество профилировать предмет типа 
хоть я и школота но поверь у мы то же самый общество профилировать предмет типа


In [ ]:
# проверял корректное отображение путей к файлу - не получалось
print(os.getcwd())
print(type(os.getcwd()))
print()
print(pathlib.Path.cwd())
print(type(pathlib.Path.cwd()))
print()
# бред какой-то с этими путями!!!
print(os.path.abspath('tweets_lemm.csv'))
print(type(os.path.abspath('tweets_lemm.csv')))
print()
print(os.path.dirname(os.path.abspath('tweets_lemm.csv')))
print(type(os.path.dirname(os.path.abspath('tweets_lemm.csv'))))

### Создание мешка слов

In [40]:
data = pd.read_csv('datasets\\tweets_lemm.csv')
data.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,вот что значит страшилка но блин посмотреть ве...


In [41]:
# преобразование столбца датасета 'lemm_text' с леммами слов в список текстов с кодировкой Unicode
corpus = data['lemm_text'].values.astype('U')
corpus

array(['хоть я и школотый но поверь у мы то же самый общество профилировать предмет тип ',
       'да весь таки он немного похожий на он но мой мальчик весь равно хороший ',
       'ну ты идиотка я испугаться за ты ', ...,
       'вот и в школа в говно это идти уже надо ',
       'тауриэль не грусть обнять ',
       'такси везти я на работа раздумывать приплатить чтобы я втащить на пять этаж лифт то нет '],
      dtype='<U139')

#### 1) мешок слов без удаления стоп-слов

In [54]:
# создание мешка слов без удаления стоп-слов
count_vect = CountVectorizer()
bow = count_vect.fit_transform(corpus)

print('Размер мешка без удаления стоп-слов:', bow.shape)
# отображение мешка слов в виде матрицы
print(bow.toarray())
# список уникальных слов в мешке
print('Список уникальных слов в мешке:', count_vect.get_feature_names_out())

Размер мешка без удаления стоп-слов: (5000, 9345)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Список уникальных слов в мешке: ['аа' 'ааа' 'аааа' ... 'ёлочка' 'ёооо' 'ёпт']


#### 2) Мешок слов с удалением стоп-слов

In [53]:
# создание мешка слов с удалением стоп-слов
# загрузка списка стоп-слов
nltk.download('stopwords')
# выделение рускоязычных слов
stop_words = stopwords.words('russian')
# создание мешка слов с исключенными стоп-словами
count_vect = CountVectorizer(stop_words=stop_words)

bow = count_vect.fit_transform(corpus)
print('Размер мешка с удаленными стоп-словами:', bow.shape)

Размер мешка с удаленными стоп-словами: (5000, 9248)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Victor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Оценка важности слов через *TF-IDF* (Term Frequency (*TF*)-Inverse Document Frequency (*IDF*))

$\begin{aligned}
&TF\text-IDF=TF*IDF\\
&TF=\frac{t}{N}\\
&IDF=log{_{10}}{\left(\frac{D}{d}\right)}\text{ ,}
\end{aligned}$

где:  
TF — частота упоминаний слова в отдельном тексте;  
IDF — частота упоминаний слова во всём корпусе;  
t — количество повторений слова в тексте;  
n — количество слов в тексте;  
D — количество текстов в корпусе;  
d — количество текстов в которых встречается слово.

In [106]:
# создание матрицы cо значениями TF-IDF
tf_idf = TfidfVectorizer(stop_words=stop_words).fit_transform(corpus)
print("Размер матрицы:", tf_idf.shape)
print('TF-IDF слов из первого текста:', tf_idf.toarray()[0][tf_idf.toarray()[0] != 0])

Размер матрицы: (5000, 9248)
TF-IDF слов из первого текста: [0.36770615 0.39488367 0.39488367 0.42854609 0.23482263 0.36122125
 0.42854609]


### Создание N-грамм

In [56]:
corpus2 = list(data['lemm_text'])
corpus2[:5]

['хоть я и школотый но поверь у мы то же самый общество профилировать предмет тип ',
 'да весь таки он немного похожий на он но мой мальчик весь равно хороший ',
 'ну ты идиотка я испугаться за ты ',
 'кто то в угол сидеть и погибать от голод а мы ещё порция взять хотя уже и так жрать не хотеть ',
 'вот что значит страшилка но блин посмотреть весь часть у ты создаться ощущение что автор курить что то ']

In [43]:
# создание n-грамм из двух слов
count_vect = CountVectorizer(ngram_range=(2, 2))
n_gramm = count_vect.fit_transform(corpus2)
print("Размер:", n_gramm.shape)

Размер: (5000, 32701)


### Примеры использования

#### Пример применения лемматизации

In [ ]:
from pymystem3 import Mystem

u_data = data['purpose'].unique()
df = pd.DataFrame(data=u_data,columns=['purpose'])

m = Mystem()
target = df['purpose'].apply(m.lemmatize)

def credit_target(target):
    if 'образование' in target:
        return 'образование'
    if 'свадьба' in target:
        return 'свадьба'
    if 'автомобиль' in target:
        return 'автомобиль'
    if 'недвижимость' or 'жилье' in target:
        return 'недвижимость'
    else:
        return 'прочее'

df['purpose_cat'] = target.apply(credit_target)
df

#### Пример обработки текста с помощью TF-IDF

In [ ]:
# Задание:
# с помощью логистической регрессии предсказать настроение твитов в тестовой выборке

# загрузка выборок
train = pd.read_csv('datasets\\tweets_lemm_train.csv')
test = pd.read_csv('datasets\\tweets_lemm_test.csv')
train.info()
display(train.head())
test.info()
display(test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5000 non-null   object
 1   positive   5000 non-null   int64 
 2   lemm_text  5000 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3000 non-null   object
 1   lemm_text  3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [ ]:
# обучение модели расчета TF-IDF на обучающей выборке
corpus_train = train['lemm_text'].values.astype('U')
mod_tf_idf = TfidfVectorizer(stop_words=stop_words).fit(corpus_train)

In [ ]:
# получение предсказаний TF-IDF на обучающей выборке, 
# которые станут признаками для логистической регрессии на обучающей выборке
features_train = mod_tf_idf.transform(corpus_train)
features_train.shape

(5000, 9737)

In [ ]:
# получение предсказаний TF-IDF на тестовой выборке, 
# которые станут признаками для логистической регрессии на тестовой выборке
corpus_test = test['lemm_text'].values.astype('U')
features_test = mod_tf_idf.transform(corpus_test)
features_test.shape

(3000, 9737)

In [ ]:
# выделение целевого признака из обучающей выборки
target_train = train['positive'].values
target_train.shape

(5000,)

In [ ]:
# обучение модели логистической регрессии на обучающей выборке
mod_lg = LogisticRegression(random_state=12345)
mod_lg.fit(features_train, target_train)

LogisticRegression(random_state=12345)

In [ ]:
# получение предсказаний логистической регрессии на тестовой выборке,
# сохранение их в отдельном столбце тестовой выборки
predict = mod_lg.predict(features_test)
print(predict.shape)
test['positive'] = predict
test.info()
test.head()

(3000,)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       3000 non-null   object
 1   lemm_text  3000 non-null   object
 2   positive   3000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 70.4+ KB


,text,lemm_text,positive
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...,1
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...,0
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...,0
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...,0
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...,1


In [ ]:
# экспорт предсказаний в файл
test['positive'].to_csv('datasets\predictions', index=False)

# конец задания
# -----------------------

#### Пример использования pipeline из transformers с предобученной моделью с Hugging Face

In [10]:
from transformers import pipeline

clf = pipeline(
    task = 'sentiment-analysis', 
    model = 'SkolkovoInstitute/russian_toxicity_classifier')

text = ['у нас есть',
    	'Как минимум два малолетних дегенерата в треде, мда.']

clf(text) # возвращает наиболее вероятный класс
# clf(text, top_k=None) # возвращает вероятности всех классов

# вариант генератора с yield для построчной загрузки в память, когда её не хватает для больших объемов данных
#def data(text):
#    for row in text:
#        yield row
#for out in clf(data(text)):
#    print(out)

[{'label': 'neutral', 'score': 0.9997156262397766},
 {'label': 'toxic', 'score': 0.985331654548645}]

#### Обработка текста в помощью ruBERT

Нужно научиться определять, какие твиты негативной тональности, какие позитивной.

Для решения этой задачи используется фреймворк PyTorch, которая применяется в задачах обработки естественного текста (NLP) и компьютерного зрения, в частности при работе с моделями типа BERT, которые находятся в библиотеке transformers. Для обучения применяется модель RuBERT из открытого репозитория DeepPavlov, обученная на разговорном русскоязычном корпусе.